#Utilities

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.preprocessing import image
import pandas as pd
import numpy as np
import cv2
import imageio
from PIL import Image
from sklearn.utils import class_weight
import numpy as np
from keras import layers
from keras import Input
from keras import utils


In [ ]:
BATCH_SIZE = 512
EPOCHS = 20
VALIDATION_SPLIT = 0.3
directory_orientation = ""
directory_light = ""
img_dim = (64,64)


In [ ]:
def get_CNN4_model(input_layer, dropout = 0):
  x = layers.Conv2D(8, (5,5), activation=None, padding="same")(input_layer)
  x = layers.Activation("relu")(x)
  x = layers.Conv2D(16, (5,5), activation=None, padding="same")(x)
  x = layers.Activation("relu")(x)
  x = layers.Conv2D(32, (5,5), activation=None, padding="same")(x)
  x = layers.Activation("relu")(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64, (5,5), activation=None, padding="same")(x)
  x = layers.Activation("relu")(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1024, activation="relu")(x)
  x = layers.Dropout(dropout)(x)
  x = layers.Dense(3, activation="softmax")(x)
  model = tf.keras.models.Model(inputs=[input_layer], outputs=x)
  model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model



#Light model training

In [ ]:
# Preparing the dataset
image_generator = image.ImageDataGenerator(rescale = 1. /255, validation_split = VALIDATION_SPLIT, preprocessing_function = None )
train_data = image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory = directory_light, subset = "training", shuffle = True, target_size = img_dim, class_mode = "categorical")
validation_data = image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory = directory_light, subset = "validation", shuffle = True, target_size = img_dim, class_mode = "categorical")
tot_train = len(train_data.classes)
tot_val = len(validation_data.classes)

# Balancing the classes
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_data.classes), 
            train_data.classes)
weight = {i : class_weights[i] for i in range(3)}
print(weight)


# Building the model
input_img = Input(shape=(*img_dim, 3), name="img")
model_light = get_CNN4_model(input_img, 0.2)
model_light.summary()
#utils.plot_model(model_light, show_shapes=True)

In [ ]:
history_light = model_light.fit(train_data, epochs=EPOCHS, class_weight=weight, steps_per_epoch=int(np.ceil(tot_train / BATCH_SIZE)), validation_data=validation_data, validation_steps=int(np.ceil(tot_val / float(BATCH_SIZE)))) 

In [ ]:
 f,(ax1,ax2) = plt.subplots(1,2)
 f.set_figheight(7)
 f.set_figwidth(14)

 ax2.step(np.arange(EPOCHS), history_light.history['accuracy'],"b")  
 ax1.step(np.arange(EPOCHS), history_light.history['loss'],"b")  
 ax2.step(np.arange(EPOCHS), history_light.history['val_accuracy'], "r")  
 ax1.step(np.arange(EPOCHS), history_light.history['val_loss'], "r") 
 ax2.hlines(1, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.75, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.5, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.25, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(1, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.75, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.5, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.25, 0, EPOCHS, colors="#CCCCCC")
 ax1.axis([0,EPOCHS-1, 0, 1.2]) 
 ax2.axis([0,EPOCHS-1, 0, 1.2]) 
 ax1.set_title('model loss')  
 ax2.set_title('model accuracy')  
 ax2.set_ylabel('accuracy')  
 ax1.set_ylabel('loss')  
 ax1.set_xlabel('epoch')
 ax2.set_xlabel("epoch")  
 ax1.legend([ 'train_loss', 'val_loss'], loc='best') 
 ax2.legend([ 'train_accuracy', 'val_accuracy'], loc='best') 


#Orientation model training

In [ ]:
# Preparing the dataset
image_generator_orientation = image.ImageDataGenerator(rescale = 1. /255, validation_split = VALIDATION_SPLIT, preprocessing_function = None )
train_data_orientation = image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory = directory_orientation, subset = "training", shuffle = True, target_size = img_dim, class_mode = "categorical")
validation_data_orientation = image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory = directory_orientation, subset = "validation", shuffle = True, target_size = img_dim, class_mode = "categorical")
tot_train = len(train_data_orientation.classes)
tot_val = len(validation_data_orientation.classes)

# Balancing the classes
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_data_orientation.classes), 
            train_data_orientation.classes)
weight = {i : class_weights[i] for i in range(3)}
print(weight)


# Building the model
input_img = Input(shape=(*img_dim, 3), name="img")
model_orientation = get_CNN4_model(input_img)
model_orientation.summary()
utils.plot_model(model_orientation, show_layer_names= False)

In [ ]:
history_orientation = model_orientation.fit(train_data_orientation, epochs=EPOCHS, class_weight=weight, steps_per_epoch=int(np.ceil(tot_train / BATCH_SIZE)), validation_data=validation_data_orientation, validation_steps=int(np.ceil(tot_val / float(BATCH_SIZE)))) 

In [ ]:
 f,(ax1,ax2) = plt.subplots(1,2)
 f.set_figheight(7)
 f.set_figwidth(14)

 ax2.step(np.arange(EPOCHS), history_orientation.history['accuracy'],"b")  
 ax1.step(np.arange(EPOCHS), history_orientation.history['loss'],"b")  
 ax2.step(np.arange(EPOCHS), history_orientation.history['val_accuracy'], "r")  
 ax1.step(np.arange(EPOCHS), history_orientation.history['val_loss'], "r") 
 ax2.hlines(1, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.75, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.5, 0, EPOCHS, colors="#CCCCCC")
 ax2.hlines(0.25, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(1, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.75, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.5, 0, EPOCHS, colors="#CCCCCC")
 ax1.hlines(0.25, 0, EPOCHS, colors="#CCCCCC")
 ax1.axis([0,EPOCHS-1, 0, 1.2]) 
 ax2.axis([0,EPOCHS-1, 0, 1.2]) 
 ax1.set_title('model loss')  
 ax2.set_title('model accuracy')  
 ax2.set_ylabel('accuracy')  
 ax1.set_ylabel('loss')  
 ax1.set_xlabel('epoch')
 ax2.set_xlabel("epoch")  
 ax1.legend([ 'train_loss', 'val_loss'], loc='best') 
 ax2.legend([ 'train_accuracy', 'val_accuracy'], loc='best') 
